# Function Calling with Structured Outputs

In [22]:
import os
from pydantic import BaseModel
import openai

openai_api_key = ""

# Define a structured output model
class MyStructuredModel(BaseModel):
    generic_response: str

# Prepare the messages and instructions
messages = [
    {"role": "system", "content": "You are a helpful assistant. Follow the response format instructions."},
    {"role": "user", "content": "Say something nice."}
]

# Send request to the LLM with a structured output format
response = openai.OpenAI(api_key=openai_api_key).beta.chat.completions.parse(
    model="gpt-4o",   # Replace with your desired model
    messages=messages,
    response_format=MyStructuredModel
)

# The parsed response will be an instance of MyStructuredModel
parsed_response = response.choices[0].message.parsed
print("Parsed Structured Response:", parsed_response)
print("Type:", type(parsed_response))
print("generic_response field:", parsed_response.generic_response)


Parsed Structured Response: generic_response="You're doing a wonderful job, and your efforts truly make a positive impact. Keep shining bright!"
Type: <class '__main__.MyStructuredModel'>
generic_response field: You're doing a wonderful job, and your efforts truly make a positive impact. Keep shining bright!


In [24]:
from typing import Optional, Dict, Any, List
from pydantic import BaseModel, Field
from typing import Literal

from typing import Optional, List, Dict, Any
from pydantic import BaseModel, Field
from typing import Literal

# Filters
class IntegerPropertyFilter(BaseModel):
    property_name: str
    operator: Literal["=", "<", ">", "<=", ">="]
    value: float  # numeric type since operator implies numeric comparison

class TextPropertyFilter(BaseModel):
    property_name: str
    operator: Literal["=", "LIKE"]
    value: str

class BooleanPropertyFilter(BaseModel):
    property_name: str
    operator: Literal["=", "!="]
    value: bool

# Aggregations
class IntegerPropertyAggregation(BaseModel):
    property_name: str
    metrics: Literal["COUNT", "TYPE", "MIN", "MAX", "MEAN", "MEDIAN", "MODE", "SUM"]

class TextPropertyAggregation(BaseModel):
    property_name: str
    metrics: Literal["COUNT", "TYPE", "TOP_OCCURRENCES"]
    top_occurrences_limit: Optional[int] = None

class BooleanPropertyAggregation(BaseModel):
    property_name: str
    metrics: Literal["COUNT", "TYPE", "TOTAL_TRUE", "TOTAL_FALSE", "PERCENTAGE_TRUE", "PERCENTAGE_FALSE"]

# This model aggregates all possible arguments for the query_database tool.
# All fields are optional so the LLM can choose which ones to include.
class ToolArguments(BaseModel):
    collection_name: str
    search_query: Optional[str] = None
    integer_property_filter: Optional[IntegerPropertyFilter] = None
    text_property_filter: Optional[TextPropertyFilter] = None
    boolean_property_filter: Optional[BooleanPropertyFilter] = None
    integer_property_aggregation: Optional[IntegerPropertyAggregation] = None
    text_property_aggregation: Optional[TextPropertyAggregation] = None
    boolean_property_aggregation: Optional[BooleanPropertyAggregation] = None
    groupby_property: Optional[str] = None

class ToolCall(BaseModel):
    function_name: str
    arguments: ToolArguments

class ResponseOrToolCalls(BaseModel):
    reflection_about_tool_use: Optional[str] = Field(
        default=None,
        description="A rationale regarding whether tool calls are needed."
    )
    use_tools: bool
    response: Optional[str] = None
    tool_calls: Optional[List[ToolCall]] = None


# Tool Description Prompt
tools_description = """
Tools:
- name: search_database
- description: Search a database with the following collections and their respective schemas:
BostonRestaurants (collection): name (text), description (searchable text), averageRating (number), and openNow (bool).
- arguments:
-- search_query (str): The query to search with.
-- integer_property_filter (object): A filter to apply to retrieve objects that match a condition.s
"""

prompt = "How many restaurants in Boston are rated higher than a 4?"

messages = [
    {"role": "system", "content": f"You are a helpful assistant. Follow the response format instructions and use the tools if needed. Here is a description of the available tools {tools_description}"},
    {"role": "user", "content": prompt}
]

response = openai.OpenAI(api_key=openai_api_key).beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    response_format=ResponseOrToolCalls
)

# The .parsed attribute should hold an instance of FunctionCall
parsed_response = response.choices[0].message.parsed

print("Parsed Response:")
print(parsed_response)

Parsed Response:
reflection_about_tool_use='I need to search the BostonRestaurants database to find the count of restaurants with an averageRating higher than 4.' use_tools=True response=None tool_calls=[ToolCall(function_name='search_database', arguments=ToolArguments(collection_name='BostonRestaurants', search_query=None, integer_property_filter=IntegerPropertyFilter(property_name='averageRating', operator='>', value=4.0), text_property_filter=None, boolean_property_filter=None, integer_property_aggregation=IntegerPropertyAggregation(property_name='averageRating', metrics='COUNT'), text_property_aggregation=None, boolean_property_aggregation=None, groupby_property=None))]


In [25]:
parsed_response.reflection_about_tool_use

'I need to search the BostonRestaurants database to find the count of restaurants with an averageRating higher than 4.'

In [26]:
parsed_response.tool_calls

[ToolCall(function_name='search_database', arguments=ToolArguments(collection_name='BostonRestaurants', search_query=None, integer_property_filter=IntegerPropertyFilter(property_name='averageRating', operator='>', value=4.0), text_property_filter=None, boolean_property_filter=None, integer_property_aggregation=IntegerPropertyAggregation(property_name='averageRating', metrics='COUNT'), text_property_aggregation=None, boolean_property_aggregation=None, groupby_property=None))]